In [54]:
# %pip show openai
%pip install --upgrade openai

In [55]:
# %pip freeze > requirements.txt

In [63]:
import json
import dotenv
import os
import openai
from openai import OpenAI
from datetime import datetime
import random

In [57]:
JSON_FILE_PATH = "data\\id_added_dummy_data.json"
RCMD_PROMPT_PATH = "..\\prompts\\openai_recommend.md"

In [58]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [91]:
# json type check
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)
    print(type(json_data))

<class 'list'>


In [105]:
def extract_workspaceId(obj, key):
    values = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == key:
                values.append(v)
            values.extend(extract_workspaceId(v, key))
    elif isinstance(obj, list):
        for item in obj:
            values.extend(extract_workspaceId(item, key))

    return values

target_key = "workspaceId"
ws_id_list = extract_workspaceId(json_data, target_key)

print(ws_id_list)
type(ws_id_list[0])

[10, 9, 12, 15, 2, 18, 16, 3, 2, 18, 4, 12, 3, 5, 8, 1, 1, 7, 9, 11, 17, 6, 2, 10, 18, 5, 2, 17, 19, 16, 1, 19, 15, 17, 6, 17, 18, 12, 19, 5, 15, 19, 10, 8, 11, 14, 12, 4, 11, 2, 3, 16, 4, 9, 10, 13, 13, 18, 7, 4, 2, 18, 14, 16, 2, 10, 20, 17, 16, 18, 19, 20, 6, 4, 10, 2, 11, 18, 1, 13, 16, 14, 5, 2, 10, 10, 17, 14, 4, 4, 11, 13, 11, 6, 2, 18, 2, 8, 3, 1]


int

In [ ]:
WORKSPACE_ID = random.choice(ws_id_list)
print("WORKSPACE_ID =", WORKSPACE_ID)

WORKSPACE_ID = 6


In [107]:
# 하나의 workspace 추출
def extract_ws(ws_id, input_data):
    for workspace in input_data:
        if workspace.get("workspaceId") == ws_id:
            return workspace
    return None

extracted_ws = extract_ws(WORKSPACE_ID, json_data)
extracted_ws

{'workspaceId': 6,
 'recommendedCategories': [{'categoryId': 4,
   'name': '게시판 기능',
   'features': [{'featureId': 81,
     'name': '관리 기능',
     'actions': [{'name': '게시글 승인',
       'importance': 3,
       'startDate': '2025-06-14T14:25:34.094309',
       'endDate': '2025-06-27T14:25:34.094309'},
      {'name': '스팸 필터링',
       'importance': 3,
       'startDate': '2025-06-15T14:25:34.094312',
       'endDate': '2025-06-28T14:25:34.094312'},
      {'name': '통계 관리',
       'importance': 5,
       'startDate': '2025-06-13T14:25:34.094316',
       'endDate': '2025-06-18T14:25:34.094316'}]},
    {'featureId': 11,
     'name': '게시글 관리',
     'actions': [{'name': '게시글 작성',
       'importance': 4,
       'startDate': '2025-06-20T14:25:34.094320',
       'endDate': '2025-06-22T14:25:34.094320'},
      {'name': '파일 첨부',
       'importance': 3,
       'startDate': '2025-06-16T14:25:34.094323',
       'endDate': '2025-06-29T14:25:34.094323'},
      {'name': '게시글 신고',
       'importance': 4,
   

In [108]:
# 워크스페이스에서 featureId 추출
def extract_featureId(obj, key):
    values = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == key:
                values.append(v)
            values.extend(extract_featureId(v, key))
    elif isinstance(obj, list):
        for item in obj:
            values.extend(extract_featureId(item, key))

    return values

target_key = "featureId"
f_id_list = extract_featureId(extracted_ws, target_key)

print(f_id_list)
type(f_id_list[0])

[81, 11, 63, 55, 4]


int

In [113]:
FEATURE_ID = random.choice(f_id_list)
print("FEATURE_ID =", FEATURE_ID)

FEATURE_ID = 55


In [114]:
def extract_view(feature_id, input_ws):
    workspaceId = input_ws.get("workspaceId")
    categories = input_ws.get("recommendedCategories", [])
    
    for category in categories:
        for feature in category.get("features", []):
            if feature.get("featureId") == feature_id:
                return {
                    "workspaceId": workspaceId,
                    "categoryId": category.get("categoryId"),
                    "featureId": feature_id,
                    "actions": feature.get("actions", [])
                }
            
            else: print(f"featureId={FEATURE_ID}에 해당하는 workspace가 없습니다.")
    
    return None

temp = extract_view(FEATURE_ID, extracted_ws)
temp

featureId=55에 해당하는 workspace가 없습니다.
featureId=55에 해당하는 workspace가 없습니다.
featureId=55에 해당하는 workspace가 없습니다.


{'workspaceId': 6,
 'categoryId': 3,
 'featureId': 55,
 'actions': [{'name': '장바구니 기능',
   'importance': 1,
   'startDate': '2025-06-13T14:25:34.094343',
   'endDate': '2025-06-26T14:25:34.094343'},
  {'name': '주문 취소 처리',
   'importance': 1,
   'startDate': '2025-06-13T14:25:34.094346',
   'endDate': '2025-06-18T14:25:34.094346'},
  {'name': '배송 추적',
   'importance': 4,
   'startDate': '2025-06-14T14:25:34.094349',
   'endDate': '2025-06-23T14:25:34.094349'}]}

In [61]:
##### test - Read task_recommend prompt #####
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    SYSTEM_PROMPT = f.read()
    print("task recommend prompt:\n", SYSTEM_PROMPT)  # test

task recommend prompt:
 # 작업 흐름 기반 액션 추천 프롬프트

## 당신의 역할

당신의 역할은 다음과 같습니다:
- 현재까지 수행된 또는 예정된 작업 흐름을 분석하고,
- 그 흐름에서 자연스럽게 이어질 다음 액션(action)을 **정확히 3가지** 제안하는 것입니다.
- 추천하는 액션은 추상적이거나 모호하지 않아야 하며,
- 구체적으로 어떤 기능에 필요한 어떤 동작을 구현해야 하는지 분명하게 명시해야 합니다.

## 중요 지침

- 하나의 feature에서 반드시 **3개 이상의 액션을 추천해야 하며**, 1개나 2개만 추천하면 실패로 간주됩니다.
- 모든 추천 액션은 **기존 feature의 actions 배열에 추가되어야 하며**, feature의 name도 명시해야 합니다.
- 액션 각각에 대해 다음 정보를 반드시 포함하세요: name, startDate, endDate, importance
- 출력은 순수 JSON 구조만 있어야 하며, 부가적인 설명, 안내 문구는 포함하지 마세요.
- 출력하는 actions는 추천하는 actions만 최종 출력해야 합니다.

---

## 요구사항

- 반드시 맥락 기반으로 전체 작업 흐름을 분석해 현실적이고 실현 가능한 다음 작업을 3가지 추천할 것
- 추천하는 action은 프로젝트 구현에 반드시 필요한 기능이어야 하며, 항상 새로운 작업을 추천해야 합니다.
- 추천하는 action은 **UI 수준의 일반 설명이 아니라, 구체적인 기술적 구현 단위**로 작성해야 합니다.
- 작업 소요 시간은 기존 작업들의 평균 기간 및 연속성 흐름을 반영하여 startDate, endDate를 현재 날짜를 기준으로 합리적으로 지정할 것
- 추천하는 작업의 중요도(importance)는 프로젝트 목표 및 기존 작업과의 연관성을 기반으로 판단할 것
- actions의 최종 출력은 추천하는 작업(actions)만 출력할 것

### 응답 형식

{
  "workspaceId": 읽어온 workspaceId,
  

In [124]:
client = OpenAI()
MODEL_ID = 'gpt-4o-mini'

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%dT%H:%M:%S")  # 날짜 형식 지정

def recommend_task(id):

    with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
        RCMD_PROMPT = f.read()
    
    # task recommend prompt -> 프로젝트 진행 단계(맥락) 파악해서 추가해야 할 작업 추천
    prompt = f"""
    ## 당신의 역할

    당신의 역할은 다음과 같습니다:
    - 현재까지 수행된 또는 예정된 작업 흐름을 분석하고, 추가적으로 구현해야 할 다음 액션(action)을 **3가지** 제안하는 것입니다.
    - 추천하는 액션은 추상적이거나 모호하지 않아야 하며,
    - 구체적으로 어떤 기능에 필요한 어떤 동작을 구현해야 하는지 분명하게 명시해야 합니다.

    ## 중요 지침
    - 하나의 feature에서 반드시 **3개 이상의 액션을 추천해야 하며**, 1개나 2개만 추천하면 실패로 간주됩니다.
    - 모든 추천 액션은 **기존 feature의 actions 배열에 추가되어야 하며**, feature의 name도 명시해야 합니다.
    - 액션 각각에 대해 다음 정보를 반드시 포함하세요: name, startDate, endDate, importance
    - 출력은 순수 JSON 구조만 있어야 하며, 부가적인 설명, 안내 문구는 포함하지 마세요.
    - 출력하는 actions는 추천하는 actions만 최종 출력해야 합니다.

    ## 요구사항
    - 반드시 맥락 기반으로 전체 작업 흐름을 분석해 현실적이고 실현 가능한 다음 작업을 3가지 추천할 것
    - 추천하는 action은 프로젝트 구현에 반드시 필요한 기능이어야 하며, 항상 새로운 작업을 추천해야 합니다.
    - 추천하는 action은 **UI 수준의 일반 설명이 아니라, 구체적인 기술적 구현 단위**로 작성해야 합니다.
     - 예: WebSocket 서버와 클라이언트 간 연결 상태 유지 로직 구현, 배송 상태 변경에 따른 자동 푸시 알림 트리거 구축
    - 작업 소요 시간은 기존 작업들의 평균 기간 및 연속성 흐름을 반영하여 startDate, endDate를 현재 날짜를 기준으로 합리적으로 지정할 것
    - 추천하는 작업의 중요도(importance)는 프로젝트 목표 및 기존 작업과의 연관성을 기반으로 판단할 것
    - actions의 최종 출력은 추천하는 작업(actions)만 출력할 것

    **중복 방지 강화 지침**
    - 이전에 이미 존재하는 actions의 name과 중복되지 않는 작업만 추천하세요.
    - 기존 작업과 동일하거나 유사한 이름/목적을 가진 작업은 절대 제안하지 마세요.
    - 매번 다른 상황을 가정하고, 유사한 기능이라도 다른 세부 구현이나 후속 단계에 초점을 맞춰 새로운 작업을 추천하세요.

    **다양성 유도 지침**
    - 항상 같은 작업을 반복 추천하지 말고, 구현 방식, 순서, 중요도, 기술 요소 등을 바꾸어 다양한 유형의 후속 작업을 제안하세요.
    - 만약 기술적으로 가능한 경우, `추가적인 연동`, `성능 개선`, `에러 처리`, `로그 수집`, `보안 강화`, `테스트 자동화`와 같은 맥락에서 파생되는 작업도 고려하세요.

    ### 응답 형식

    {{
        "workspaceId": 읽어온 workspaceId,
        "categoryId": 읽어온 categoryId,
        "featureId": 읽어온 featureId,
        "recommendedActions": [
            {{
            "name": "추천하는 action"
            "importance": "추천 actions의 중요도(1~5 사이의 값)",
            "startDate": "LocalDateTime형식의 날짜",
            "endDate": "LocalDateTime형식의 날짜"
            }},
        ]
    }}

    작업 흐름은 다음과 같습니다:
    {extracted_ws}
    """

    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": RCMD_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    rcmd_task = response.choices[0].message.content.strip()

    return rcmd_task

print(recommend_task(extracted_ws))

{
  "workspaceId": 6,
  "categoryId": 4,
  "featureId": 81,
  "recommendedActions": [
    {
      "name": "게시글 수정",
      "importance": 4,
      "startDate": "2025-06-28T14:25:34.094309",
      "endDate": "2025-07-01T14:25:34.094309"
    },
    {
      "name": "게시글 삭제",
      "importance": 3,
      "startDate": "2025-07-02T14:25:34.094309",
      "endDate": "2025-07-05T14:25:34.094309"
    },
    {
      "name": "댓글 관리",
      "importance": 4,
      "startDate": "2025-07-06T14:25:34.094309",
      "endDate": "2025-07-09T14:25:34.094309"
    }
  ]
}
